## General Usage

In [ ]:
%load_ext autoreload
%autoreload 2
%pprint off

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%pprint

### Preamble

[!] use only if you haven't installed FHIR PACK via `pip`, `pipenv` or similar in your environment but you have instead cloned the project and are now in the `/examples` directory

In [ ]:
import sys,os
sys.path.append(os.getcwd()+'/../src/')
# sys.path

### Imports

In [ ]:
import fhirpack			as fp
import pandas			as pd
import fhirpy
import numpy			as np
import fhir.resources	as fr
import json
from tqdm import tqdm
from fhirpack.constants import CONFIG

### Useful Paths for Data Storage 

In [ ]:
DATAPATH=CONFIG.get('DATAPATH')
RESPATH=f"{DATAPATH}/fhir/"

### FHIR Server Connection

### CLI Usage

In [ ]:
!dotenv -f ../.env run python -m fhirpack.cli "-o" "getPatients" -p "given = Chalmers" #-v

!fp -o "getPatients" -p "given = Chalmers" #-v

### From Custom Environment File

In [ ]:
!head ../.env.example -n5

In [ ]:
pack= fp.PACK()

for i,e in pack.getPatients(['2918556']).data.items():
	print(
		json.dumps(e.serialize(), indent=4, sort_keys=True)[:200],
		'\n...'
	)

### Manual Setup

In [ ]:
pack  = fp.PACK("http://hapi.fhir.org/baseR4")
pack.getPatients(['2918556'])

In [ ]:
pack  = fp.PACK("http://test.fhir.org/r4/")
pack.getPatients(['11']).data.iloc[0]

In [ ]:
#run "docker-compose up" in your local clone of FHIRPACK first  

# pack  = fp.PACK("http://127.0.0.1:42112/hapi-fhir-jpaserver/fhir")
# pack.getPatients(['1']).data.iloc[0].serialize()


### From Default Environment File

In [ ]:
pack = fp.PACK()
# pack.client.authorization

### Introductory Example

In [ ]:
# for patients whose lastname is Koepp, find the root identity,
# get their associated diagnostic reports and report their ID and status

pack.getPatients(searchParams={"family":"koepp"})\
	.getRootPatients()\
		.explode()\
			.getDiagnosticReports()\
				.dropna()\
					.gatherSimplePaths(['id','status'])\
						.explode(['id','status'])

### FHIR PACK Usage 

### References

In [ ]:
patientReference = pack.getReferences(['Patient/647487'])

patientReference.data.iloc[0].to_resource().serialize()

patientReference.getResources()

### Resources

### Reference to Resource

In [ ]:
multiPatientReference = pack.getReferences(
    [
        'Patient/1849165',
        'Patient/647487'
    ])

multiPatientReference.data.apply( lambda x:x.id )

### Direct Resource

In [ ]:
pack.getPatients(searchParams={"family":"Betterhalf"})

In [ ]:
pack.getPatients(searchParams={}).size

In [ ]:
multiPatientResource = pack.getResources(
    ['Patient/1849165',
     'Patient/647487']
)

multiPatientResource

### Serialization and Element Access

In [ ]:
patientResource = patientReference.data.iloc[0].to_resource()

patientResource['name'][0]['given']

patientResource.serialize().get('birthDate')

In [ ]:
multiPatientReference.getResources()

In [ ]:
multiPatientResource[:1].pretty

In [ ]:
fp.PACK().getDiagnosticReports(
    ['Patient/647487']
)[:1].keys


### DataFrame Operations

In [ ]:
multiPatientReference[:1]

multiPatientReference[-1:]

multiPatientReference.values

multiPatientReference.info()

### pack.base.Frame

In [ ]:
fp.base.Frame( [[patientResource]], columns=['data'])

### Extraction

### Patient Extraction

### fhirpack.extraction.getPatients

In [ ]:
pack.getPatients(
	[
		'647487'
	]
)


In [ ]:
pack.getPatients(
	[
		'647487'
	]
).data.item().serialize()

In [ ]:
for i,e in pack.getPatients(['647487','1849165']).itertuples():
    print (f"Row {i} with Patient ID {e.id}")

In [ ]:
getPatientsAsList=pack.getPatients(['647487','1849165']).cast('list')

getPatientsAsList

In [ ]:
[e.pop().id for e in getPatientsAsList]

### Patients as Operand and Extracting other Resources 

In [ ]:
patients = pack.getPatients(
    ['9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
    '1849165',
    '649231',
    '650224']
)


### fhirpack.extraction.getRootPatient

In [ ]:
pack.getRootPatients(
    [
        'b558da74-7756-4845-87d5-d1cca8b79a62',
    ],
).explode().gatherSimplePaths(["id"]).id.unique()

In [ ]:
patients.getRootPatients()

In [ ]:
pack.getResources(
	[
		'Patient/b558da74-7756-4845-87d5-d1cca8b79a62'
	]
).getRootPatients()

In [ ]:
rootPatients = patients.getResources().getRootPatients().explode()

rootPatients.gatherSimplePaths(['id','birthDate'])

In [ ]:
rootPatients.data[0].name

In [ ]:
for link in rootPatients.data[0]['link']:
	lpat=link.other.to_resource()
	print(lpat.id)

### fhirpack.extraction.getLinkedPatients

In [ ]:
pack.getLinkedPatients(
	[
		'9ac622c4-61c8-40b4-b6d9-06cfeb3d6995',
	]
)

### fhirpack.extraction.getConditions

### Conditions

In [ ]:
conditions = pack.getConditions(
	searchParams={
		'code':'44465007',
		'_sort': '-onset-date',
	}
)[:5]

conditions

### Patients for Conditions

In [ ]:
conditions.getPatients().explode().gatherSimplePaths(['id','name.given','name.family'])


In [ ]:
conditions\
	.getPatients()\
		.explode()\
			.data.apply(lambda x:x.id)\
				.to_csv(f"data.ignore")

In [ ]:
!cat data.ignore

In [ ]:
conditions = pd.read_csv( f"data.ignore", index_col=0)
conditions.shape
conditions

### Conditions for Patient

In [ ]:
pack.getPatients(['8cbf1128-3644-47a1-9cc8-05f1aac6071d']).getConditions().explode()

### fhipack.extraction.getEpisodesOfCare

In [ ]:
rootPatients[:2]

In [ ]:
pack.getPatients(['P0522-patientBSJ1']).getEpisodesOfCare()

### fhipack.extraction.getFamilyMemberHistories

In [ ]:
pack.getPatients(['Patient/2866670']).getFamilyMemberHistories()

### fhipack.extraction.getMedicationAdministrations

In [ ]:
pack.getPatients(['Patient/31678']).getMedicationAdministrations()[:5]

### fhipack.extraction.getMedicationRequests

In [ ]:
pack.getPatients(['Patient/202205uscore-patient-example-1']).getMedicationRequests().explode()

### fhipack.extraction.getObservations

In [ ]:
pack\
	.getPatients(['Patient/258974'])\
		.getConditions(searchParams={"code":"44465007"})\
			.explode()\
				.getPatients(searchParams={"family":"Keebler762"})

In [ ]:

from datetime import datetime

interesting = pack.getPatients(['Patient/2164033']).\
    getObservations(
        searchParams={
            "code":"http ://loinc.org|29463-7",
            }
    )\
        .gatherSimplePaths(['id','issued','code.coding.code'])\
            .explode(['id','issued','code.coding.code'])

interesting.issued = pd.to_datetime(interesting.issued).dropna().apply(lambda x:x.replace(tzinfo=None))
interesting[interesting.issued> datetime.fromisoformat('2021-01-01T00:00:00')]

### fhirpack.extraction.getDiagnosticReports

In [ ]:
diagnosticReports=pack\
	.getPatients(["Patient/9b8c1901-62ee-48a7-8229-b771d59f1e5f"])\
		.getDiagnosticReports(
			searchParams={}
			)\
			.explode()

diagnosticReports

In [ ]:
paths=[
	"subject",
	"presentedForm.contentType",
	"presentedForm.data",
	"presentedForm.url",
	"presentedForm.title",
	"presentedForm.creation"
]



diagnosticReports=pack.getDiagnosticReports(
	searchParams={
		"_id":"19bdf90a-8ca4-4921-8aeb-2bf3423aaf09",
		# "identifier":"|",
		# "_content":"covid19",
		# "code":"|",
		# "issued__gt":"2010-01-01",
		# "issued__lt":"2011-01-01"
	}
)

diagnosticReports.gatherSimplePaths(paths)

### fhirpack.extraction.getURLBytes

In [ ]:
paths=[
    'id',
    'subject.reference',
    'presentedForm.url'
]

diagnosticReports = pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
    )\
        .getDiagnosticReports()[:10]

diagnosticReports=diagnosticReports.gatherSimplePaths(paths)
diagnosticReports=diagnosticReports.explode('presentedForm.url')
diagnosticReports

# diagnosticReports['path']=diagnosticReports['id']+'_'+diagnosticReports['presentedForm.url'].str.split('/').str[-1:].str[0]
# diagnosticReports['data']=diagnosticReports.getURLBytes(operateOnCol='presentedForm.url')
# diagnosticReports.sendBytesToFile()

In [ ]:
diagnosticReports.loc[:, diagnosticReports.columns!='data'][:5]

### fhirpack.extraction.base.getAbsolutePaths

In [ ]:
absolutePaths=[
#    'Condition.code.coding.code',
   'Condition.id',
    'Condition.subject.reference',
    'Encounter.id',
    'Encounter.participant.individual.reference',
#     'Procedure.code.coding.display',
#     'Procedure.code.coding.code',
    'Procedure.id',
    'Procedure.status',
    'Procedure.performedDateTime'
]

result=pack.getPatients(
    ['Patient/e92abcdc-a300-41e0-aa0f-378daebe25dc']
).getAbsolutePaths(absolutePaths)

In [ ]:
result['Procedure']

In [ ]:
paths=[
	'valueQuantity.value',
	'valueQuantity.unit',
	'valueQuantity.system',
	'valueQuantity.code',
]

In [ ]:
pack.\
	getPatients(['e92abcdc-a300-41e0-aa0f-378daebe25dc'])\
		.getObservations()\
			.explode()\
				.gatherSimplePaths(paths)

In [ ]:
result["Condition"]

In [ ]:
result["Encounter"]

In [ ]:
result["Procedure"]

In [ ]:
result['Encounter'].explode(
	'Encounter.participant.individual.reference'
)\
	.rename(
		columns={"Encounter.participant.individual.reference":"data"}
		)\
			.dropna()\
				.getResources()[:5]\
					.gatherSimplePaths(['name.given','name.family'])

### fhirpack.extraction.getImagingStudies

In [ ]:
conds=pack.getConditions(
	searchParams={
		"code":"C61",
		# "_content": "leber"
	}
	)
conds

In [ ]:
pack\
	.getImagingStudies(searchParams={"endpoint:missing":False})\
		.gatherSimplePaths(['series.description','series.uid','identifier.value'])

# use .getDICOMInstances().sendDICOMToFiles() to download DICOM files

### Transformation

### fhirpack.transformation.base.gatherKeys

In [ ]:
patients.gatherKeys().data.explode().value_counts()[:10]

In [ ]:
pack.getPatients(
    ['e92abcdc-a300-41e0-aa0f-378daebe25dc']
    )\
        .gatherKeys(['valueString'])\
            .explode()

### fhirpack.transformation.base.valuesForKeys

In [ ]:
pack.getPatients(
    ['e92abcdc-a300-41e0-aa0f-378daebe25dc'],
    # includeLinkedPatients=True
    )\
        .gatherValuesForKeys(['valueString'])\
            .explode()

### fhirpack.transformation.base.gatherReferences

In [ ]:
pack.gatherReferences(
    ['Patient/9ac622c4-61c8-40b4-b6d9-06cfeb3d6995']
    , recursive=True
)

### fhirpack.transformation.base.gatherText

In [ ]:
fp.PACK().getPatients(
    ['Patient/1555106'],
    )\
        .getObservations()[:5]\
            .explode()\
                .gatherText()

### fhirpack.transformation.base.gatherDates

In [ ]:
fp.PACK().getPatients(
    ['Patient/1555106'],
    )\
        .getObservations()[:5]\
            .explode()\
                .gatherValuesForKeys(['issued'])

In [ ]:
fp.PACK().getPatients(
    ['Patient/1555106'],
    )\
        .getObservations()[:5]\
            .explode()\
                .gatherDates()

### Load

In [ ]:
pack.validate(
    ['Patient/1555106']
    )\
        .sendResourcesToFiles(['./data.ignore'])

### Utils

### Server Profiling

In [ ]:
pack.countServerResources()

### Validation

In [ ]:
pack.validate(
    ['Patient/1555106']
)


### FHIR PACK Internals

#### FHIRPy

### URL Parsing

In [ ]:
import fhirpy.base.utils as fpu
url=fpu.parse_pagination_url('/app/FHIR/r4/Condition?code=C61&_count=1&_sort=-onset-date&identifier=%7C&_Pagination=eyJvZmZzZXQiOjIwfQ%3D%3D')
url

### Raw fetch_resource()

In [ ]:
pack.client._fetch_resource(
	"Condition/1499041"
)

### Raw client.execute()

In [ ]:
pack.client.execute('/baseR4/Condition',
method='get',
params={
	'code':'C61',
	'_sort':'-onset-date',
	'_count': 1,
	'_total':'accurate'
}
).total

### Customization

### fhirpack.custom.extraction.base

In [ ]:
# the fhirpack.custom package is guaranteed to never be used by us
# use it to customize fhirpack

# pack.unimplementedPluginBaseExtractorMethod()
# pack.unimplementedPluginBaseTransformerMethod()
# pack.unimplementedPluginBaseLoaderMethod()

# pack.unimplementedPluginSampleExtractorMethod()
# pack.unimplementedPluginSampleTransformerMethod()
# pack.unimplementedPluginSampleLoaderMethod()